In [1]:
import math
import numpy as np
import random
from collections import Counter
from scipy import stats
from tqdm import tqdm
from scipy import stats

In [2]:
from os import walk
mypath = '../data/'
filenames = next(walk(mypath), (None, None, []))[2] 

NUM_DIGITS = 1000000
NUM_RUNS = 1000
pi_cf_digits = []
for filename in tqdm(filenames):
    pi_digits = np.load(mypath+filename)
    pi_cf_digits.extend(pi_digits['arr_0'])
pi_cf_digits = pi_cf_digits[1:(NUM_DIGITS*NUM_RUNS) +1]
pi_cf_digits = pi_cf_digits[:math.floor((len(pi_cf_digits)/NUM_DIGITS))* NUM_DIGITS]
pi_cf_digits = [int(x) for x in pi_cf_digits]
len(pi_cf_digits)

100%|█████████████████████████████████████████████| 3/3 [00:45<00:00, 15.13s/it]


1000000000

# Normalized distance

### Random Walk

In [3]:
nums = [math.log(1+1/(k*(k+2)))/math.log(2) for k in range(1, NUM_DIGITS+1)]
distribution = [0 for i in range(4)]
for i in range(len(nums)):
    distribution[i%4] += nums[i]
inv_distribution = [1/distribution[i] for i in range(len(distribution))]
normalized_inv_distribution = [float(i)/sum(inv_distribution) for i in inv_distribution]
moves = [(1*inv_distribution[0], 0),
         (0, 1*inv_distribution[1]),
         (-1*inv_distribution[2], 0),
         (0, -1*inv_distribution[3])]

In [4]:
moves

[(2.000001442695982, 0),
 (0, 4.18055196959628),
 (-6.600844419128332, 0),
 (0, -9.149106876384758)]

In [5]:
distribution

[0.49999963932626473,
 0.23920286298858545,
 0.15149576879923704,
 0.10930028619308763]

In [6]:
def dist_from_origin(coordinates):
    return math.sqrt((coordinates[0]**2) + (coordinates[1]**2))

In [7]:
def single_random_walk_run(random_dist):
    random_walk = [(0, 0)]
    for i in rand_distribution:
        random_walk.append((random_walk[-1][0] + moves[i%4][0], random_walk[-1][1] + moves[i%4][1]))
    distance = 0
    for i in random_walk[1:]:
        distance += dist_from_origin(i)
    avg_dist = distance/(NUM_DIGITS * 500*math.pi)
    return avg_dist

In [8]:
avg_distances = []
for i in tqdm(range(NUM_RUNS)):
    rand_distribution = random.choices([0, 1, 2, 3], weights = distribution, k = NUM_DIGITS)
    avg_distances.append(single_random_walk_run(rand_distribution))

100%|███████████████████████████████████████| 1000/1000 [12:42<00:00,  1.31it/s]


In [9]:
stats.describe(avg_distances)

DescribeResult(nobs=1000, minmax=(0.525199098305482, 5.20868758913382), mean=1.7712678264263217, variance=0.5376952457471398, skewness=1.0203981005403895, kurtosis=1.2553391883861575)

### $\pi$ Walk

In [10]:
groups = list(np.array_split(pi_cf_digits, NUM_RUNS))

In [11]:
def pi_walk(pi_digits):
    mod_4_digits = [(i-1)%4 for i in pi_digits]
    pi_walk = [(0, 0)]
    for i in range(len(mod_4_digits)):
        pi_walk.append((pi_walk[-1][0] + moves[mod_4_digits[i]][0], pi_walk[-1][1] + moves[mod_4_digits[i]][1]))
    total_distance = 0
    for i in range(len(pi_walk)):
        total_distance += dist_from_origin(pi_walk[i])
    return total_distance/(NUM_DIGITS * 500*math.pi)

In [12]:
avg_pi_distances = []
test = []
for i in tqdm(range(NUM_RUNS)):
    avg_pi_distances.append(pi_walk(groups[i]))

100%|███████████████████████████████████████| 1000/1000 [15:52<00:00,  1.05it/s]


In [13]:
stats.describe(avg_pi_distances)

DescribeResult(nobs=1000, minmax=(0.561931025874709, 4.682126995385607), mean=1.7467031852067483, variance=0.5131872149904297, skewness=0.8894734961739355, kurtosis=0.7972315617386845)

# Sites visited

### Random Walk

In [14]:
nums = [math.log(1+1/(k*(k+2)))/math.log(2) for k in range(1, NUM_DIGITS+1)]
distribution = [0 for i in range(4)]
for i in range(len(nums)):
    distribution[i%4] += nums[i]
inv_distribution = [1/distribution[i] for i in range(len(distribution))]
normalized_inv_distribution = [float(i)/np.average(inv_distribution) for i in inv_distribution]
moves = [(1*normalized_inv_distribution[0], 0),
         (0, 1*normalized_inv_distribution[1]),
         (-1*normalized_inv_distribution[2], 0),
         (0, -1*normalized_inv_distribution[3])]

In [15]:
moves

[(0.3647889493367027, 0),
 (0, 0.7625090302838979),
 (-1.2039566817226974, 0),
 (0, -1.6687453386567015)]

In [16]:
def single_random_walk_run(random_dist):
    random_walk = [(0, 0)]
    for i in rand_distribution:
        random_walk.append((random_walk[-1][0] + moves[i%4][0], random_walk[-1][1] + moves[i%4][1]))
    sites = set()
    for i in random_walk:
        sites.add((math.floor(i[0]), math.floor(i[1])))
    return len(sites)

In [17]:
sites_visited = []
for i in tqdm(range(NUM_RUNS)):
    rand_distribution = random.choices([0, 1, 2, 3], weights = distribution, k = NUM_DIGITS)
    sites_visited.append(single_random_walk_run(rand_distribution))

100%|███████████████████████████████████████| 1000/1000 [15:52<00:00,  1.05it/s]


In [18]:
stats.describe(sites_visited)

DescribeResult(nobs=1000, minmax=(90854, 176387), mean=147905.047, variance=134260828.44123223, skewness=-0.6297412032768313, kurtosis=0.9257049837853848)

### $\pi$ Walk

In [19]:
groups = list(np.array_split(pi_cf_digits, NUM_RUNS))

In [20]:
def pi_walk(pi_digits):
    mod_4_digits = [(i-1)%4 for i in pi_digits]
    pi_walk = [(0, 0)]
    for i in range(len(mod_4_digits)):
        pi_walk.append((pi_walk[-1][0] + moves[mod_4_digits[i]][0], pi_walk[-1][1] + moves[mod_4_digits[i]][1]))
    sites = set()
    for i in pi_walk:
        sites.add((math.floor(i[0]), math.floor(i[1])))
    return len(sites)

In [21]:
sites_visited_pi = []
for i in tqdm(range(NUM_RUNS)):
    sites_visited_pi.append(pi_walk(groups[i]))

100%|███████████████████████████████████████| 1000/1000 [15:14<00:00,  1.09it/s]


In [22]:
stats.describe(sites_visited_pi)

DescribeResult(nobs=1000, minmax=(97143, 172690), mean=146080.107, variance=137343287.3529039, skewness=-0.48425332725778186, kurtosis=0.17904975432847792)